In [ ]:
alpha = 0.1 # risk limit
name = "Election name"
winners = 1 # number of winners

In [ ]:
results = [17800, 9700] # ballots for candidates
total = 0 # total number of ballots cast (can be set to 0)

In [ ]:
#pip3 install git+git://github.com/filipzz/athena.git

In [1]:
import sys
import argparse
import string
import math
import logging
from athena.athena import AthenaAudit
from athena.election import Election
from athena.audit import Audit

In [ ]:
audit_type = "athena" 

In [ ]:
logging.getLogger(__name__).setLevel("INFO")

In [ ]:
delta = 1
candidates = [string.ascii_uppercase[i] for i in range(len(results))]
pstop_goal = []
round_schedule = []
ballots_cast = total
if ballots_cast < sum(results):
    ballots_cast = sum(results)
model = "bin"
election = {}
election["ballots_cast"] = ballots_cast
election["alpha"] = alpha
election["delta"] = delta
election["candidates"] = candidates
election["results"] = results
election["ballots_cast"] = ballots_cast
election["winners"] = winners
election["name"] = name
election["model"] = model
election["pstop"] = pstop_goal
election["round_schedule"] = round_schedule
election_object = Election(election)
election_object.print_election()

In [ ]:
audit_completed = False

while audit_completed is False:
    print("\n\nYour choices: ")
    print("[1] Find next round size at 70%, 80%, 90%")
    print("[2] Enter other goal for probability of stopping.")
    choice = input("Enter your choice: ")
    if choice == "1":
        pstop_goal = [.7, .8, .9]
    elif choice == "2":
        pstop_choice = float(input("Enter value: "))
        if 0 < pstop_choice < 1:
            pstop_goal = [pstop_choice]
        elif 1 <= pstop_choice <= 99:
            pstop_goal = [pstop_choice/100]
        else:
            print("Entered value is incorrect")
            sys.exit(1)
    else:
        audit_completed = True

    w = Audit(audit_type, alpha, delta)
    w.add_election(election)
    w.add_round_schedule(round_schedule)
    x = w.find_next_round_size(pstop_goal)
    #print(str(x))

    future_round_sizes = x["future_round_sizes"]

    if len(round_schedule) > 0:
        below_kmin = max(required) - max(observed)
        n_future_round_sizes =  list(map(lambda x: x - max(round_schedule) + 2 * below_kmin, future_round_sizes))
    else:
        n_future_round_sizes = future_round_sizes

    print("\nYour round sizes choices are: ")
    for p, rs in zip(pstop_goal, n_future_round_sizes):
        print("Probability of with %s when you sample %s more ballots." % (p, rs))


    del w

    print("\n\nEnter number of ballots drawn: ")
    new_total = int(input("Number of ballots: "))
    print("\nEnter number of ballots for the winner: ")
    new_winner = int(input("Ballots for winner: "))

    if len(round_schedule) > 0:
        round_schedule = round_schedule + [new_total + max(round_schedule)]
        actual_kmins = actual_kmins + [new_winner + max(actual_kmins)]
    else:
        round_schedule = [new_total]
        actual_kmins = [new_winner]

    w = Audit(audit_type, alpha, delta)
    w.add_election(election)
    w.add_round_schedule(round_schedule)
    x = w.find_risk(actual_kmins)
    observed = x["observed"]
    required = x["required"]

    if x["passed"] == 1:
        audit_completed = True
        print("\n\nAudit Successfully completed!")
        print("P-value:\t%s" % (x["risk"]))
    else:
        print("\n\nAnother round is required:")
        print("P-value:\t%s" % (x["risk"]))
    #print(str(x))
    del w